In [105]:
import pandas as pd
from rapidfuzz import process
from sentence_transformers import SentenceTransformer, util
import torch

In [147]:
df = pd.read_csv("C:\\Users\\waila\\Desktop\\Datasets\\Published news for December 2024 CSV.csv")

In [109]:
# 1 - exact match method
def exact_match(input_title):
    matches = df[df["Title"] == input_title]
    if matches.empty:
        return "No exact match found."
    return [{"Title": row["Title"], "Link": row["Link"]} for _, row in matches.iterrows()]


In [111]:
input_title1 = "ع / رياضي / مدرب المنتخب السعودي : الخسارة أمام عمان مسؤوليتنا بالكامل ونحتاج لتحسين دفاعنا"
exact_match(input_title1)

[{'Title': 'ع / رياضي / مدرب المنتخب السعودي : الخسارة أمام عمان مسؤوليتنا بالكامل ونحتاج لتحسين دفاعنا',
  'Link': 'https://spa.gov.sa/w2236008'}]

In [113]:
def fuzzy_match(input_title, threshold=90):
    titles = df["Title"].tolist()
    match, score, idx = process.extractOne(input_title, titles)
    if score >= threshold:
        result = df.iloc[idx]
        return {"Title": result["Title"], "Link": result["Link"]}
    return "No fuzzy match found."

In [149]:
input_title2 = " الخسارة أمام عمان"
fuzzy_match(input_title2)

{'Title': 'ع / رياضي / مدرب المنتخب السعودي : الخسارة أمام عمان مسؤوليتنا بالكامل ونحتاج لتحسين دفاعنا',
 'Link': 'https://spa.gov.sa/w2236008'}

In [155]:
def semantic_match(input_title, top_n=5, threshold=0.6):
  
    input_embedding = model.encode(input_title, convert_to_tensor=True)
    
    all_embeddings = torch.stack(df["embedding"].tolist())
    
    cosine_scores = util.cos_sim(input_embedding, all_embeddings)[0]
    
    valid_indices = torch.where(cosine_scores >= threshold)[0]
    
    if valid_indices.numel() == 0:
        return "No matching titles found."
    
    sorted_valid_indices = valid_indices[torch.argsort(cosine_scores[valid_indices], descending=True)]
    
    top_indices = sorted_valid_indices[:top_n]
    
    results = df.iloc[top_indices.cpu().numpy()].copy()
    
    results["similarity_score"] = cosine_scores[top_indices].cpu().numpy()
    
    formatted_results = results[["Title", "Link"]].to_dict(orient="records")
    
    return formatted_results

In [157]:
if "embedding" not in df.columns:
    df["embedding"] = df["Title"].apply(lambda x: model.encode(x, convert_to_tensor=True))

input_title3 = "السعوديه ضد عمان"
top_matches = semantic_match(input_title3)

if isinstance(top_matches, str):
    print(top_matches)
else:
    print("\nTop Matching Titles:\n")
    for match in top_matches:
        print(f"Title: {match['Title']}\nLink: {match['Link']}\n")



Top Matching Titles:

Title: ع / رياضي / الفارس الأردني عابورة: "قفز السعودية" أصداؤها تتجاوز النطاق المحلي
Link: https://spa.gov.sa/w2221358

Title: ع / سياحة وترفيه / "موسم الرياض": مواجهة كبيرة ضمن POWER SLAP للمرة الأولى في السعودية 
Link: https://spa.gov.sa/w2220681

Title: ع / ثقافي / "عام الإبل".. تظاهرة فريدة أبرزت عمق ارتباط السعوديين بالإبل
Link: https://spa.gov.sa/w2234152

Title: ع / رياضي / مدرب المنتخب السعودي : الخسارة أمام عمان مسؤوليتنا بالكامل ونحتاج لتحسين دفاعنا
Link: https://spa.gov.sa/w2236008

Title: ع / رياضي / مواجهة حاسمة تجمع المنتخب السعودي والمنتخب عماني غدًا في نصف نهائي "خليجي 26"
Link: https://spa.gov.sa/w2235507

